In [ ]:
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: percent
#       format_version: '1.3'
#       jupytext_version: 1.3.4
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# %%
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import csv

# --- Custom Activation Functions ---
def custom_sechlu(x, rho=1.0):
    """
    Implementiert eine Sigmoid-weighted Linear Unit basierend auf der Formel:
    f(x) = x / (1 + exp(-2x / rho))

    Diese Funktion skaliert die Eingabe x mit einer Sigmoid-ähnlichen Kurve.
    Der Parameter rho steuert die Steilheit der Kurve um den Ursprung.

    Argumente:
        x (tf.Tensor): Der Eingabe-Tensor.
        rho (float): Ein anpassbarer Parameter zur Steuerung der Form.
                     Standardwert ist 1.0.

    Rückgabe:
        tf.Tensor: Der Tensor nach Anwendung der Aktivierungsfunktion.
    """
    # Die Formel kann direkt mit TensorFlow-Operationen umgesetzt werden.
    # tf.sigmoid(y) ist mathematisch äquivalent zu 1 / (1 + exp(-y)).
    # Hier ist y = 2x / rho.
    return x * tf.sigmoid(2 * x / rho)

def custom_cauchylu(x, rho=1.0):
    """
    Implementiert eine Cauchy-basierte Aktivierungsfunktion mit der Formel:
    f(x) = x/2 * (1 + 2/pi * arctan(x/rho))

    Diese Funktion skaliert die Eingabe auf eine komplexe, nicht-lineare Weise.
    Der Parameter rho passt die Breite der zentralen Region an.

    Argumente:
        x (tf.Tensor): Der Eingabe-Tensor.
        rho (float): Ein anpassbarer Parameter zur Steuerung der Form.
                     Standardwert ist 1.0.

    Rückgabe:
        tf.Tensor: Der Tensor nach Anwendung der Aktivierungsfunktion.
    """
    # Wir setzen die Formel Schritt für Schritt um

    # 1. Berechne den inneren Teil: x / rho
    scaled_x = x / rho

    # 2. Berechne den Arcustangens
    arctan_x = tf.math.atan(scaled_x)

    # 3. Berechne den gesamten Skalierungsfaktor
    gating_factor = 0.5 * (1.0 + (2.0 / np.pi) * arctan_x)

    # 4. Multipliziere mit der ursprünglichen Eingabe x
    #    (Hinweis: Die Formel wurde leicht umgestellt von x/2 * (...) zu x * (0.5 * (...))
    #    für eine klarere Implementierung, das Ergebnis ist identisch)
    return x * gating_factor

def custom_laplacelu(x, rho=1.0):
    """
    Implementiert eine Laplace-basierte Aktivierungsfunktion mit der Formel:
    f(x, rho) = x/2 * (1 + sgn(x) * (1 - exp(-abs(x)/rho)))

    Diese Funktion skaliert die Eingabe x mit einem Faktor, der von der
    kumulativen Verteilungsfunktion der Laplace-Verteilung inspiriert ist.
    Der Parameter rho steuert die Steilheit der Kurve.

    Argumente:
        x (tf.Tensor): Der Eingabe-Tensor.
        rho (float): Ein anpassbarer Parameter zur Steuerung der Form.
                     Standardwert ist 1.0.

    Rückgabe:
        tf.Tensor: Der Tensor nach Anwendung der Aktivierungsfunktion.
    """
    # Wir setzen die Formel direkt mit TensorFlow-Funktionen um.

    # 1. Berechne den Absolutwert von x
    abs_x = tf.abs(x)

    # 2. Berechne den exponentiellen Term
    exp_term = tf.exp(-abs_x / rho)

    # 3. Berechne den von der Vorzeichenfunktion abhängigen Teil
    sign_term = tf.sign(x) * (1.0 - exp_term)

    # 4. Berechne den gesamten Skalierungsfaktor ("Gate")
    gating_factor = 0.5 * (1.0 + sign_term)

    # 5. Multipliziere mit der ursprünglichen Eingabe x
    #    (Umstellung von x/2 * (...) zu x * (0.5 * (...)) für Klarheit)
    return x * gating_factor

# It's also good practice to register them with Keras if they are going to be used by string name directly in layers
# However, for this task, we will pass the function objects directly.

# --- 1. Daten vorbereiten ---
def generate_damped_oscillator_data(num_samples=1000,
                                   amplitude=1.0,
                                   decay_constant=0.5,
                                   frequency=2.0,
                                   phase=0.0,
                                   noise_amplitude=0.00):
    """
    Generiert synthetische Daten für einen gedämpften harmonischen Oszillator.

    Args:
        num_samples (int): Anzahl der zu generierenden Datenpunkte.
        amplitude (float): Anfangsamplitude des Oszillators.
        decay_constant (float): Die Abklingkonstante (Gamma).
        frequency (float): Die Winkelfrequenz (Omega).
        phase (float): Die Phasenverschiebung.
        noise_amplitude (float): Amplitude des hinzugefügten Rauschens.

    Returns:
        tuple: Ein Tupel von (time_steps, positions)
    """
    time_steps = np.linspace(0, 10, num_samples) # Zeit von 0 bis 10
    true_positions = amplitude * np.exp(-decay_constant * time_steps) * \
                     np.cos(frequency * time_steps + phase)
    # Rauschen hinzufügen, um das Modell robuster zu machen
    noise = noise_amplitude * np.random.randn(num_samples)
    positions = true_positions + noise
    return time_steps, positions

# Generieren der Daten
time_data, position_data = generate_damped_oscillator_data()

# Daten für Keras vorbereiten (Input muss 2D sein, Output kann 1D sein)
# reshape(-1, 1) stellt sicher, dass es eine Spalte und beliebig viele Zeilen hat
X = time_data.reshape(-1, 1)
y = position_data

# Daten aufteilen in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape von X_train: {X_train.shape}")
print(f"Shape von y_train: {y_train.shape}")
print(f"Shape von X_test: {X_test.shape}")
print(f"Shape von y_test: {y_test.shape}")

# %%
# --- Main Experiment Loop ---
neuron_configurations = [[32, 32], [32, 16], [16, 32]]
# Custom activation functions (custom_sechlu, custom_cauchylu, custom_laplacelu) are already defined globally.

activation_function_map = {
    'relu': tf.keras.activations.relu,
    'gelu': tf.keras.activations.gelu,
    'tanh': tf.keras.activations.tanh,
    # mish requires TensorFlow >= 2.3 (tf.keras.activations.mish) or from tf_addons.
    # Assuming the execution environment supports tf.keras.activations.mish.
    'mish': tf.keras.activations.mish,
    'sechlu': custom_sechlu,
    'cauchylu': custom_cauchylu,
    'laplacelu': custom_laplacelu
}
activation_function_names = list(activation_function_map.keys())

results_list = []

# --- 2. Modell erstellen (Anzahl der Schichten und Neuronen) ---
# Note: build_oscillator_model is defined below this block, which is fine.

def build_oscillator_model(layer1_neurons,
                           layer1_activation_func,
                           layer2_neurons,
                           layer2_activation_func,
                           learning_rate=0.001,
                           input_shape=(1,)):
    """
    Erstellt ein Keras-Modell für die Vorhersage eines gedämpften Oszillators.

    Args:
        layer1_neurons (int): Anzahl der Neuronen in der ersten Schicht.
        layer1_activation_func (callable or str): Aktivierungsfunktion für die erste Schicht.
        layer2_neurons (int): Anzahl der Neuronen in der zweiten Schicht.
        layer2_activation_func (callable or str): Aktivierungsfunktion für die zweite Schicht.
        learning_rate (float): Die Lernrate für den Adam-Optimierer.
        input_shape (tuple): Input-Shape für die erste Schicht.

    Returns:
        keras.Model: Das kompilierte Keras-Modell.
    """
    model = keras.Sequential()

    # Erste Dense Schicht
    model.add(layers.Dense(layer1_neurons, activation=layer1_activation_func, input_shape=input_shape))

    # Zweite Dense Schicht
    model.add(layers.Dense(layer2_neurons, activation=layer2_activation_func))

    # Ausgabeschicht
    model.add(layers.Dense(1, activation='linear'))

    # --- 3. Optimierer konfigurieren ---
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    # --- 4. Modell kompilieren ---
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

    return model

# --- Konfiguration der Modellparameter (für Einfachheit und Präzision) ---
# Für "einfachst" und "höchste Präzision" müssen wir ein wenig experimentieren.
# Beginnen wir mit einem sehr einfachen Setup und passen es bei Bedarf an.

# Parameter für die Modellarchitektur
# Diese Werte werden nun direkt beim Aufruf von build_oscillator_model übergeben.
# Beispiel:
# model = build_oscillator_model(layer1_neurons=32, layer1_activation_func='relu',
#                                layer2_neurons=32, layer2_activation_func='relu',
#                                learning_rate=0.001)


# Erstelle das Modell - Beispielaufruf (muss angepasst werden, falls Training direkt hier erfolgen soll)
# Die alten Parameter NUM_HIDDEN_LAYERS, NEURONS_PER_LAYER, ACTIVATION_FUNCTION, LEARNING_RATE
# sind nicht mehr direkt für build_oscillator_model relevant in dieser Form.
# Stattdessen werden die Parameter direkt übergeben.
# Für den Moment kommentieren wir die Modellerstellung und model.summary() aus,
# da die Parameter nicht mehr auf die alte Weise definiert sind.
# model = build_oscillator_model(num_hidden_layers=NUM_HIDDEN_LAYERS,
#                                neurons_per_layer=NEURONS_PER_LAYER,
#                                activation_function=ACTIVATION_FUNCTION,
#                                learning_rate=LEARNING_RATE)

# Zeige eine Zusammenfassung des Modells (Parameteranzahl ist hier sichtbar)
# model.summary()

# %%
# --- Main Experiment Loop (continued) ---
for neurons_config in neuron_configurations:
    layer1_neurons, layer2_neurons = neurons_config
    for act_name1 in activation_function_names:
        for act_name2 in activation_function_names:
            act_func1 = activation_function_map[act_name1]
            act_func2 = activation_function_map[act_name2]

            model_name = f"{layer1_neurons}{act_name1.upper()}{layer2_neurons}{act_name2.upper()}"

            print(f"Building and training model: {model_name}...")

            model = build_oscillator_model(
                layer1_neurons=layer1_neurons,
                layer1_activation_func=act_func1,
                layer2_neurons=layer2_neurons,
                layer2_activation_func=act_func2
            )

            num_params = model.count_params()

            early_stopping_callback = EarlyStopping(
                monitor='val_loss',
                patience=50,  # As per user's original code
                restore_best_weights=True
            )

            # Ensure X_train, y_train, X_test, y_test are available from the data prep step
            history = model.fit(
                X_train, y_train,
                epochs=2000, # As per user's original code
                validation_data=(X_test, y_test),
                batch_size=32,
                callbacks=[early_stopping_callback],
                verbose=0  # Keep output clean during the loop
            )

            actual_epochs = len(history.history['loss'])

            # Evaluate the model on the test set
            loss, mae = model.evaluate(X_test, y_test, verbose=0)
            mse = loss  # model.compile uses 'mean_squared_error' as the loss (already updated)

            results_list.append([model_name, num_params, actual_epochs, mae, mse])
            print(f"Completed: {model_name} | Params: {num_params} | Epochs: {actual_epochs} | MAE: {mae:.4f} | MSE: {mse:.4f}")

            # Optional: Clear session to free memory if many models are trained
            # tf.keras.backend.clear_session()
            # Note: clear_session() also clears custom object registry, so use with care if custom objects are not re-registered.
            # For this case, it might be okay as models are built fresh each time.
            # Let's not add clear_session() for now to avoid potential complexities unless memory issues arise.

# --- Save results to TSV ---
output_tsv_file = 'model_comparison_results.tsv'
print(f"Writing results to {output_tsv_file}...")

with open(output_tsv_file, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t') # Using tab as delimiter

    # Write the header
    writer.writerow(['Name of model', 'parameters', 'epochs', 'mae', 'mse'])

    # Write the data rows
    for row in results_list: # Assuming results_list is populated by the experiment loop
        writer.writerow(row)

print(f"Results successfully saved to {output_tsv_file}.")

# %%
# --- (Old sections for single model training, evaluation, and plotting are now replaced by the loop above) ---
# --- 5. Modell trainieren ---
# ... (code removed / commented out) ...
#
# --- 6. Modell evaluieren ---
# ... (code removed / commented out) ...
#
# --- Vorhersagen machen und visualisieren ---
# ... (code removed / commented out) ...
#
# Optional: Trainingsverlauf plotten
# ... (code removed / commented out) ...

Shape von X_train: (800, 1)
Shape von y_train: (800,)
Shape von X_test: (200, 1)
Shape von y_test: (200,)
Building and training model: 32RELU32RELU...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 

Building and training model: 32RELU32RELU...
Completed: 32RELU32RELU | Params: 1153 | Epochs: 70 | MAE: 0.0896 | MSE: 0.0172
Building and training model: 32RELU32GELU...
Completed: 32RELU32GELU | Params: 1153 | Epochs: 678 | MAE: 0.0042 | MSE: 0.0000
Building and training model: 32RELU32TANH...
Completed: 32RELU32TANH | Params: 1153 | Epochs: 974 | MAE: 0.0025 | MSE: 0.0000
Building and training model: 32RELU32MISH...
Completed: 32RELU32MISH | Params: 1153 | Epochs: 1521 | MAE: 0.0015 | MSE: 0.0000
Building and training model: 32RELU32SECHLU...
Completed: 32RELU32SECHLU | Params: 1153 | Epochs: 1136 | MAE: 0.0041 | MSE: 0.0000
Building and training model: 32RELU32CAUCHYLU...
Completed: 32RELU32CAUCHYLU | Params: 1153 | Epochs: 999 | MAE: 0.0033 | MSE: 0.0000
Building and training model: 32RELU32LAPLACELU...
Completed: 32RELU32LAPLACELU | Params: 1153 | Epochs: 608 | MAE: 0.0060 | MSE: 0.0001
Building and training model: 32GELU32RELU...
Completed: 32GELU32RELU | Params: 1153 | Epochs: 1900 | MAE: 0.0032 | MSE: 0.0000
Building and training model: 32GELU32GELU...
Completed: 32GELU32GELU | Params: 1153 | Epochs: 869 | MAE: 0.0023 | MSE: 0.0000
Building and training model: 32GELU32TANH...
Completed: 32GELU32TANH | Params: 1153 | Epochs: 938 | MAE: 0.0034 | MSE: 0.0000
Building and training model: 32GELU32MISH...
Completed: 32GELU32MISH | Params: 1153 | Epochs: 954 | MAE: 0.0056 | MSE: 0.0001
Building and training model: 32GELU32SECHLU...
Completed: 32GELU32SECHLU | Params: 1153 | Epochs: 711 | MAE: 0.0043 | MSE: 0.0000
Building and training model: 32GELU32CAUCHYLU...
Completed: 32GELU32CAUCHYLU | Params: 1153 | Epochs: 1403 | MAE: 0.0047 | MSE: 0.0000
Building and training model: 32GELU32LAPLACELU...
Completed: 32GELU32LAPLACELU | Params: 1153 | Epochs: 1138 | MAE: 0.0048 | MSE: 0.0000
Building and training model: 32TANH32RELU...
Completed: 32TANH32RELU | Params: 1153 | Epochs: 1032 | MAE: 0.0057 | MSE: 0.0001
Building and training model: 32TANH32GELU...
Completed: 32TANH32GELU | Params: 1153 | Epochs: 904 | MAE: 0.0041 | MSE: 0.0000
Building and training model: 32TANH32TANH...
Completed: 32TANH32TANH | Params: 1153 | Epochs: 1043 | MAE: 0.0040 | MSE: 0.0000
Building and training model: 32TANH32MISH...
Completed: 32TANH32MISH | Params: 1153 | Epochs: 992 | MAE: 0.0059 | MSE: 0.0001
Building and training model: 32TANH32SECHLU...
Completed: 32TANH32SECHLU | Params: 1153 | Epochs: 581 | MAE: 0.0045 | MSE: 0.0000
Building and training model: 32TANH32CAUCHYLU...
Completed: 32TANH32CAUCHYLU | Params: 1153 | Epochs: 930 | MAE: 0.0038 | MSE: 0.0000
Building and training model: 32TANH32LAPLACELU...
Completed: 32TANH32LAPLACELU | Params: 1153 | Epochs: 1480 | MAE: 0.0045 | MSE: 0.0000
Building and training model: 32MISH32RELU...
Completed: 32MISH32RELU | Params: 1153 | Epochs: 372 | MAE: 0.0066 | MSE: 0.0001
Building and training model: 32MISH32GELU...
Completed: 32MISH32GELU | Params: 1153 | Epochs: 822 | MAE: 0.0030 | MSE: 0.0000
Building and training model: 32MISH32TANH...
Completed: 32MISH32TANH | Params: 1153 | Epochs: 735 | MAE: 0.0043 | MSE: 0.0000
Building and training model: 32MISH32MISH...
Completed: 32MISH32MISH | Params: 1153 | Epochs: 879 | MAE: 0.0045 | MSE: 0.0000
Building and training model: 32MISH32SECHLU...
Completed: 32MISH32SECHLU | Params: 1153 | Epochs: 479 | MAE: 0.0041 | MSE: 0.0000
Building and training model: 32MISH32CAUCHYLU...
Completed: 32MISH32CAUCHYLU | Params: 1153 | Epochs: 766 | MAE: 0.0054 | MSE: 0.0000
Building and training model: 32MISH32LAPLACELU...
Completed: 32MISH32LAPLACELU | Params: 1153 | Epochs: 1363 | MAE: 0.0010 | MSE: 0.0000
Building and training model: 32SECHLU32RELU...
Completed: 32SECHLU32RELU | Params: 1153 | Epochs: 796 | MAE: 0.0051 | MSE: 0.0001
Building and training model: 32SECHLU32GELU...
Completed: 32SECHLU32GELU | Params: 1153 | Epochs: 606 | MAE: 0.0057 | MSE: 0.0001
Building and training model: 32SECHLU32TANH...
Completed: 32SECHLU32TANH | Params: 1153 | Epochs: 929 | MAE: 0.0023 | MSE: 0.0000
Building and training model: 32SECHLU32MISH...
Completed: 32SECHLU32MISH | Params: 1153 | Epochs: 1196 | MAE: 0.0048 | MSE: 0.0000
Building and training model: 32SECHLU32SECHLU...
Completed: 32SECHLU32SECHLU | Params: 1153 | Epochs: 824 | MAE: 0.0052 | MSE: 0.0000
Building and training model: 32SECHLU32CAUCHYLU...
Completed: 32SECHLU32CAUCHYLU | Params: 1153 | Epochs: 1001 | MAE: 0.0051 | MSE: 0.0000
Building and training model: 32SECHLU32LAPLACELU...
Completed: 32SECHLU32LAPLACELU | Params: 1153 | Epochs: 929 | MAE: 0.0081 | MSE: 0.0001
Building and training model: 32CAUCHYLU32RELU...
Completed: 32CAUCHYLU32RELU | Params: 1153 | Epochs: 1825 | MAE: 0.0051 | MSE: 0.0000
Building and training model: 32CAUCHYLU32GELU...
Completed: 32CAUCHYLU32GELU | Params: 1153 | Epochs: 1295 | MAE: 0.0024 | MSE: 0.0000
Building and training model: 32CAUCHYLU32TANH...
Completed: 32CAUCHYLU32TANH | Params: 1153 | Epochs: 1659 | MAE: 0.0011 | MSE: 0.0000
Building and training model: 32CAUCHYLU32MISH...
Completed: 32CAUCHYLU32MISH | Params: 1153 | Epochs: 589 | MAE: 0.0075 | MSE: 0.0001
Building and training model: 32CAUCHYLU32SECHLU...
Completed: 32CAUCHYLU32SECHLU | Params: 1153 | Epochs: 1181 | MAE: 0.0032 | MSE: 0.0000
Building and training model: 32CAUCHYLU32CAUCHYLU...
Completed: 32CAUCHYLU32CAUCHYLU | Params: 1153 | Epochs: 675 | MAE: 0.0041 | MSE: 0.0000
Building and training model: 32CAUCHYLU32LAPLACELU...
Completed: 32CAUCHYLU32LAPLACELU | Params: 1153 | Epochs: 1106 | MAE: 0.0047 | MSE: 0.0000
Building and training model: 32LAPLACELU32RELU...
Completed: 32LAPLACELU32RELU | Params: 1153 | Epochs: 517 | MAE: 0.0043 | MSE: 0.0000
Building and training model: 32LAPLACELU32GELU...
Completed: 32LAPLACELU32GELU | Params: 1153 | Epochs: 649 | MAE: 0.0026 | MSE: 0.0000
Building and training model: 32LAPLACELU32TANH...
Completed: 32LAPLACELU32TANH | Params: 1153 | Epochs: 1110 | MAE: 0.0024 | MSE: 0.0000
Building and training model: 32LAPLACELU32MISH...
Completed: 32LAPLACELU32MISH | Params: 1153 | Epochs: 1655 | MAE: 0.0019 | MSE: 0.0000
Building and training model: 32LAPLACELU32SECHLU...
Completed: 32LAPLACELU32SECHLU | Params: 1153 | Epochs: 566 | MAE: 0.0044 | MSE: 0.0000
Building and training model: 32LAPLACELU32CAUCHYLU...
Completed: 32LAPLACELU32CAUCHYLU | Params: 1153 | Epochs: 843 | MAE: 0.0045 | MSE: 0.0000
Building and training model: 32LAPLACELU32LAPLACELU...
Completed: 32LAPLACELU32LAPLACELU | Params: 1153 | Epochs: 994 | MAE: 0.0055 | MSE: 0.0000
Building and training model: 32RELU16RELU...
Completed: 32RELU16RELU | Params: 609 | Epochs: 151 | MAE: 0.0394 | MSE: 0.0038
Building and training model: 32RELU16GELU...
Completed: 32RELU16GELU | Params: 609 | Epochs: 961 | MAE: 0.0020 | MSE: 0.0000
Building and training model: 32RELU16TANH...
Completed: 32RELU16TANH | Params: 609 | Epochs: 914 | MAE: 0.0022 | MSE: 0.0000
Building and training model: 32RELU16MISH...
Completed: 32RELU16MISH | Params: 609 | Epochs: 1138 | MAE: 0.0051 | MSE: 0.0000
Building and training model: 32RELU16SECHLU...
Completed: 32RELU16SECHLU | Params: 609 | Epochs: 730 | MAE: 0.0053 | MSE: 0.0000
Building and training model: 32RELU16CAUCHYLU...
Completed: 32RELU16CAUCHYLU | Params: 609 | Epochs: 1198 | MAE: 0.0039 | MSE: 0.0000
Building and training model: 32RELU16LAPLACELU...
Completed: 32RELU16LAPLACELU | Params: 609 | Epochs: 1562 | MAE: 0.0015 | MSE: 0.0000
Building and training model: 32GELU16RELU...
Completed: 32GELU16RELU | Params: 609 | Epochs: 793 | MAE: 0.0081 | MSE: 0.0001
Building and training model: 32GELU16GELU...
Completed: 32GELU16GELU | Params: 609 | Epochs: 876 | MAE: 0.0027 | MSE: 0.0000
Building and training model: 32GELU16TANH...
Completed: 32GELU16TANH | Params: 609 | Epochs: 1082 | MAE: 0.0046 | MSE: 0.0000
Building and training model: 32GELU16MISH...
Completed: 32GELU16MISH | Params: 609 | Epochs: 1772 | MAE: 0.0044 | MSE: 0.0000
Building and training model: 32GELU16SECHLU...
Completed: 32GELU16SECHLU | Params: 609 | Epochs: 494 | MAE: 0.0054 | MSE: 0.0000
Building and training model: 32GELU16CAUCHYLU...
Completed: 32GELU16CAUCHYLU | Params: 609 | Epochs: 1372 | MAE: 0.0035 | MSE: 0.0000
Building and training model: 32GELU16LAPLACELU...
Completed: 32GELU16LAPLACELU | Params: 609 | Epochs: 1148 | MAE: 0.0062 | MSE: 0.0001
Building and training model: 32TANH16RELU...
Completed: 32TANH16RELU | Params: 609 | Epochs: 463 | MAE: 0.0154 | MSE: 0.0004
Building and training model: 32TANH16GELU...
Completed: 32TANH16GELU | Params: 609 | Epochs: 1010 | MAE: 0.0052 | MSE: 0.0000
Building and training model: 32TANH16TANH...
Completed: 32TANH16TANH | Params: 609 | Epochs: 1196 | MAE: 0.0040 | MSE: 0.0000
Building and training model: 32TANH16MISH...
Completed: 32TANH16MISH | Params: 609 | Epochs: 1213 | MAE: 0.0060 | MSE: 0.0001
Building and training model: 32TANH16SECHLU...
Completed: 32TANH16SECHLU | Params: 609 | Epochs: 718 | MAE: 0.0064 | MSE: 0.0001
Building and training model: 32TANH16CAUCHYLU...
Completed: 32TANH16CAUCHYLU | Params: 609 | Epochs: 721 | MAE: 0.0048 | MSE: 0.0000
Building and training model: 32TANH16LAPLACELU...
Completed: 32TANH16LAPLACELU | Params: 609 | Epochs: 943 | MAE: 0.0060 | MSE: 0.0001
Building and training model: 32MISH16RELU...
Completed: 32MISH16RELU | Params: 609 | Epochs: 672 | MAE: 0.0086 | MSE: 0.0001
Building and training model: 32MISH16GELU...
Completed: 32MISH16GELU | Params: 609 | Epochs: 802 | MAE: 0.0028 | MSE: 0.0000
Building and training model: 32MISH16TANH...
Completed: 32MISH16TANH | Params: 609 | Epochs: 917 | MAE: 0.0020 | MSE: 0.0000
Building and training model: 32MISH16MISH...
Completed: 32MISH16MISH | Params: 609 | Epochs: 906 | MAE: 0.0047 | MSE: 0.0000
Building and training model: 32MISH16SECHLU...
Completed: 32MISH16SECHLU | Params: 609 | Epochs: 869 | MAE: 0.0033 | MSE: 0.0000
Building and training model: 32MISH16CAUCHYLU...
Completed: 32MISH16CAUCHYLU | Params: 609 | Epochs: 879 | MAE: 0.0042 | MSE: 0.0000
Building and training model: 32MISH16LAPLACELU...
Completed: 32MISH16LAPLACELU | Params: 609 | Epochs: 1365 | MAE: 0.0029 | MSE: 0.0000
Building and training model: 32SECHLU16RELU...
Completed: 32SECHLU16RELU | Params: 609 | Epochs: 473 | MAE: 0.0089 | MSE: 0.0001
Building and training model: 32SECHLU16GELU...
